In [43]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import csv
import os
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Data Extraction

In [44]:
def scrape_url(url):
  try:
    res=requests.get(url,timeout=10)
    if res.status_code==200:
      soup=BeautifulSoup(res.text,'html.parser')
      if soup.title:
        title=soup.title.string.strip()
      else:
        print("No Title Found")
      content=soup.find('article')

      if content:
        content_text=content.get_text(separator=' ').strip()
      else:
        print("No Content Found")

      return title,content_text

    else:
      return "Error",f"Failed to fetch content. Status code: {res.status_code}"

  except Exception as e:
    return "Error",f"Error occurred: {str(e)}"

In [45]:
def clean_text(text):
    text=text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [46]:
df=pd.read_excel('/content/Input.xlsx')
results=[]
for _, row in df.iterrows():
  url_id=row['URL_ID']
  url=row['URL']
  title,content=scrape_url(url)

  cleaned_content=clean_text(content)
  results.append({"URL_ID":url_id,"URL":url,"Title":title,"Cleaned Content":cleaned_content})

In [47]:
results[1]

{'URL_ID': 'Netclan20241018',
 'URL': 'https://insights.blackcoffer.com/enhancing-front-end-features-and-functionality-for-improved-user-experience-and-dashboard-accuracy-in-partner-hospital-application/',
 'Title': 'Enhancing Front-End Features and Functionality for Improved User Experience and Dashboard Accuracy in Partner Hospital Application | Blackcoffer Insights',
 'Cleaned Content': 'home our success stories enhancing frontend features and functionality for improved user experience and dashboard accuracy our success stories healthcare enhancing frontend features and functionality for improved user experience and dashboard accuracy in partner hospital application by ajay bidyarthy august 26 2024 10415 client background client a leading hospital chain in the usa industry type healthcare products services healthcare solutions organization size 200 the problem build a web application develop deploy and maintain the system in the background and there will be more hospitals partnering

In [48]:
def process_urls(input,output_folder):
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

output_folder = '/content'
process_urls(input, output_folder)

csv_file_path=os.path.join(output_folder,'processed_urls.csv')

with open(csv_file_path,'w',newline='',encoding='utf-8') as csvfile:
  filenames=['URL_ID','URL','Title','Cleaned Content']
  writer=csv.DictWriter(csvfile,fieldnames=filenames)

  writer.writeheader()
  for i in results:
    writer.writerow(i)

print(f"Processed URLs have been saved to {csv_file_path}")




Processed URLs have been saved to /content/processed_urls.csv


In [49]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## cleaning data

In [10]:
import zipfile

In [50]:
def extract_from_zip(zip_path):
    word_set = set()
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if filename.endswith('.txt'):
                with zip_ref.open(filename) as file:
                    content = file.read().decode('latin-1')
                    for line in content.splitlines():
                        word = line.strip().lower()
                        if word and not word.startswith(('#', '//')):
                            word_set.add(word)
    return word_set

In [40]:
stopwords_set = extract_from_zip('/content/StopWords.zip')


In [88]:
len(stopwords_set)

12768

In [84]:
def prepare_sentiment_dictionaries(stopwords_zip, master_dict_zip):

    stopwords_set = extract_from_zip(stopwords_zip)
    positive_set = set()
    negative_set = set()

    with zipfile.ZipFile(master_dict_zip, 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if filename.lower().endswith('positive-words.txt'):
                with zip_ref.open(filename) as file:
                    for line in file.read().decode('latin-1').splitlines():
                        word = line.strip().lower()
                        if word and not word.startswith(('#', '//')) and word not in stopwords_set:
                            positive_set.add(word)

            elif filename.lower().endswith('negative-words.txt'):
                with zip_ref.open(filename) as file:
                    for line in file.read().decode('latin-1').splitlines():
                        word = line.strip().lower()
                        if word and not word.startswith(('#', '//')) and word not in stopwords_set:
                            negative_set.add(word)

    return stopwords_set, positive_set, negative_set



In [85]:
stopwords_set, positive_set, negative_set = prepare_sentiment_dictionaries('/content/StopWords.zip',
                                                                           '/content/MasterDictionary .zip')

Polarity →

```
polarity
=
(positive count  − negative count)/
 (positive count  +  negative count)

```





Gives a range from -1 (very negative) to +1 (very positive).

Subjectivity →
```
subjectivity
=
(positive count +  negative count)/
(total words without stopwords)

```



Higher value means more opinionated text.

Sentiment Score →
You can define as

``` (positive count - negative count) ```

 (a raw strength measure).

In [90]:
def filter_text(content, stop_words):
    words = word_tokenize(content)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words

In [91]:
def analyze_sentiment(words, positive_words, negative_words):
    positive_count = sum(1 for word in words if word.lower() in positive_words)
    negative_count = sum(1 for word in words if word.lower() in negative_words)
    polarity = (positive_count - negative_count) / ((positive_count + negative_count))
    subjectivity = (positive_count + negative_count) / ((len(words)))
    sentiment_score = positive_count - negative_count
    return {
        "positive_count": positive_count,
        "negative_count": negative_count,
        "polarity": polarity,
        "subjectivity": subjectivity,
        "sentiment_score": sentiment_score
    }



## Sentimental Analysis

In [93]:
def process(input_file, stop_words, positive_words, negative_words):
    df = pd.read_csv(input_file)
    results = []
    for index, row in df.iterrows():
        content = row.get("Cleaned Content")
        if pd.notnull(content):
          filtered_words = filter_text(content, stop_words)
          sentiment = analyze_sentiment(filtered_words, positive_words, negative_words)

          results.append({
                "URL_ID": row['URL_ID'],
                "positive_count": sentiment["positive_count"],
                "negative_count": sentiment["negative_count"],
                "polarity": sentiment["polarity"],
                "subjectivity": sentiment["subjectivity"],
                "sentiment_score": sentiment["sentiment_score"]
            })

    return results

In [95]:
input_file = '/content/processed_urls.csv'
output_csv_file = '/content/sentiment_analysis_results.csv'

results = process(input_file, stopwords_set,positive_set, negative_set)

In [96]:
with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["URL_ID", "positive_count", "negative_count", "polarity", "subjectivity", "sentiment_score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Sentiment analysis results saved to {output_csv_file}")

Sentiment analysis results saved to /content/sentiment_analysis_results.csv


## Analysis of Readability


In [97]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import download
download('punkt')
download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [98]:
def count_syllables(word):
    word = word.lower()
    vowels = "aeiou"
    count = 0
    prev_char_was_vowel = False
    for char in word:
        if char in vowels:
            if not prev_char_was_vowel:
                count += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False
    if word.endswith("e") and count > 1:
        count -= 1
    return count


In [99]:
def compute_readability_metrics(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    num_sentences = len(sentences)
    num_words = len(words)

    complex_words = [i for i in words if count_syllables(i) >= 3]
    num_complex_words = len(complex_words)

    average_sentence_length = num_words / num_sentences if num_sentences > 0 else 0

    percentage_complex_words = (num_complex_words / num_words) * 100 if num_words > 0 else 0

    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)


    return average_sentence_length, percentage_complex_words, fog_index


In [109]:
input_file = '/content/processed_urls.csv'
output_file = '/content/readability_metrics.csv'

df = pd.read_csv(input_file)
results = []
for _, row in df.iterrows():
    content = row.get("Cleaned Content")
    if pd.notnull(content):
        avg_sentence_length, pct_complex_words, fog_idx = compute_readability_metrics(content)
        results.append({
            "URL_ID": row.get("URL_ID"),  # Ensure URL_ID is preserved
            "Average Sentence Length": avg_sentence_length,
            "Percentage of Complex Words": pct_complex_words,
            "Fog Index": fog_idx
        })

In [101]:
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["URL_ID", "Average Sentence Length", "Percentage of Complex Words", "Fog Index"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Readability metrics saved to {output_file}")

Readability metrics saved to /content/readability_metrics.csv


## Average Number of Words Per Sentence

In [102]:
def calculate_metrics(text):
    words = re.findall(r'\b\w+\b', text)
    total_words = len(words)

    sentences = re.split(r'[.!?]+', text)
    total_sentences = len([s for s in sentences if s.strip()])

    average_words_per_sentence = total_words / total_sentences if total_sentences else 0

    return total_words, total_sentences, average_words_per_sentence

In [105]:
input_file = '/content/processed_urls.csv'
output_file = '/content/average_words_per_sentence.csv'

df = pd.read_csv(input_file)
results = []

for _, row in df.iterrows():
    content = row.get("Cleaned Content")
    if pd.notnull(content):
        total_words, total_sentences, avg_words_per_sentence = calculate_metrics(content)

        results.append({
            "URL_ID": row.get("URL_ID"),
            "Total Words": total_words,
            "Total Sentences": total_sentences,
            "Average Words Per Sentence": avg_words_per_sentence
        })

In [106]:
output_df = pd.DataFrame(results)
output_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Metrics saved to {output_file}")


Metrics saved to /content/average_words_per_sentence.csv


## complex count

In [111]:
def complex_words(text):
    words = word_tokenize(text)
    complex_words = [w for w in words if count_syllables(w) >= 3]
    return len(complex_words)

In [112]:
input_file = '/content/processed_urls.csv'
output_file = '/content/complex_word_count.csv'

df = pd.read_csv(input_file)
results = []

for _, row in df.iterrows():
    content = row.get("Cleaned Content")
    if pd.notnull(content):
        complex_word_count = complex_words(content)
        results.append({
            "URL_ID": row.get("URL_ID"),
            "Complex Word Count": complex_word_count
        })


In [113]:
output_df = pd.DataFrame(results)
output_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Complex word count results saved to {output_file}")


Complex word count results saved to /content/complex_word_count.csv


## word count

In [117]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def count_cleaned_words(text):
    words = word_tokenize(text)
    cleaned_words = [re.sub(r'[^\w\s]', '', word.lower()) for word in words if word.lower() not in stop_words]
    cleaned_words = [word for word in cleaned_words if word]
    return len(cleaned_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
input_file = '/content/processed_urls.csv'
output_file = '/content/word_count_results.csv'

df = pd.read_csv(input_file)
results = []

for _, row in df.iterrows():
    content = row.get("Cleaned Content")
    if pd.notnull(content):
        word_count = count_cleaned_words(content)
        results.append({
            "URL_ID": row.get("URL_ID"),
            "Cleaned Word Count": word_count
        })

In [119]:
output_df = pd.DataFrame(results)
output_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Word count results saved to {output_file}")


Word count results saved to /content/word_count_results.csv


## count syllables

In [121]:
def count_syllables(word):
    word = word.lower()
    vowels = "aeiou"
    syllable_count = len(re.findall(r'[aeiou]+', word))
    if word.endswith("es") or word.endswith("ed"):
        if len(word) > 2 and not re.search(r'[aeiou]', word[:-2]):
            syllable_count -= 1
    return max(1, syllable_count)

def analyze_syllables(text):
    words = re.findall(r'\b\w+\b', text)
    syllable_counts = [count_syllables(word) for word in words]
    return sum(syllable_counts), syllable_counts

In [122]:
input_file = '/content/processed_urls.csv'
output_file = '/content/syllable_count_results.csv'

df = pd.read_csv(input_file)
results = []

for _, row in df.iterrows():
    content = row.get("Cleaned Content")
    if pd.notnull(content):
        total_syllables, syllable_list = analyze_syllables(content)
        results.append({
            "URL_ID": row.get("URL_ID"),
            "Total Syllables": total_syllables,
            "Average Syllables Per Word": total_syllables / len(syllable_list) if syllable_list else 0
        })


In [123]:
output_df = pd.DataFrame(results)
output_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Syllable count results saved to {output_file}")


Syllable count results saved to /content/syllable_count_results.csv


## count personal pronouns

In [124]:
def count_personal_pronouns(text):
    pronoun_pattern = r'\b(I|we|my|ours|us)\b'
    matches = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)
    matches = [i for i in matches if i.lower() != "us"]
    return len(matches)

In [125]:
input_file = "/content/processed_urls.csv"
output_file = "/content/personal_pronouns_analysis.csv"

df = pd.read_csv(input_file)

df['Personal Pronouns Count'] = df['Cleaned Content'].apply(count_personal_pronouns)

df.to_csv(output_file, index=False)

print(f"Personal pronouns analysis saved to {output_file}")

Personal pronouns analysis saved to /content/personal_pronouns_analysis.csv


## average word length

In [126]:
def calculate_average_word_length(text):
    words = re.findall(r'\b\w+\b', text)
    total_characters = sum(len(word) for word in words)
    total_words = len(words)
    if total_words == 0:
        return 0

    return total_characters / total_words

In [127]:
input_file = "/content/processed_urls.csv"
df['Average Word Length'] = df['Cleaned Content'].apply(calculate_average_word_length)

output_file = "/content/average_word_length_analysis.csv"
df.to_csv(output_file, index=False)

print(f"Average word length analysis saved to {output_file}")

Average word length analysis saved to /content/average_word_length_analysis.csv


In [128]:
sentiment_file = '/content/sentiment_analysis_results.csv'
readability_file = '/content/readability_metrics.csv'
avg_words_file = '/content/average_words_per_sentence.csv'
complex_word_count_file = '/content/complex_word_count.csv'
word_count_file = '/content/word_count_results.csv'
syllable_count_file = '/content/syllable_count_results.csv'
average_word_length_file = '/content/average_word_length_analysis.csv'

In [137]:
df_sentiment = pd.read_csv(sentiment_file)
df_readability = pd.read_csv(readability_file)
df_avg_words = pd.read_csv(avg_words_file)
df_complex_word= pd.read_csv(complex_word_count_file)
df_word_count = pd.read_csv(word_count_file)
df_syllable_count = pd.read_csv(syllable_count_file)
df_average_word_length=pd.read_csv(average_word_length_file)



In [147]:
combined_df = df_sentiment.merge(df_readability, on="URL_ID", how="outer")
combined_df = combined_df.merge(df_avg_words, on="URL_ID", how="outer")
combined_df= combined_df.merge(df_complex_word, on="URL_ID", how="outer")
combined_df = combined_df.merge(df_word_count, on="URL_ID", how="outer")
combined_df = combined_df.merge(df_syllable_count, on="URL_ID", how="outer")
combined_df = combined_df.merge(df_average_word_length, on="URL_ID", how="outer")

In [148]:
output_combined = '/content/final_combined_results.csv'


In [149]:
cols = ["URL_ID", "URL", "Title","Cleaned Content"] + [c for c in combined_df.columns if c not in ["URL_ID", "URL", "Title","Cleaned Content"]]
combined_df = combined_df[cols]


In [150]:
combined_df.to_csv(output_combined, index=False, encoding='utf-8')

print(f"Combined results saved to {output_combined}")

Combined results saved to /content/final_combined_results.csv
